In [1]:
import pandas as pd 
import numpy as np 

import re 

Start by importing the data and adding in some moving averages and exponential moving averages 

In [2]:
df = pd.read_csv('FTSE 100 - 1984-2020.csv')
df = df.reindex(index=df.index[::-1])
columns = df.columns
df.head()

,Date,Open,High,Low,Close_,AdjClose__,Volume
9519,"Jan 03, 1984",997.50,"1,001.40",997.50,997.50,997.50,NaN
9518,"Jan 04, 1984",997.50,999.50,993.30,998.60,998.60,NaN
9517,"Jan 05, 1984","1,007.10","1,015.80","1,007.10","1,015.80","1,015.80",NaN
9516,"Jan 06, 1984","1,019.00","1,029.30","1,019.00","1,029.00","1,029.00",NaN
9515,"Jan 09, 1984","1,030.60","1,035.40","1,030.60","1,034.60","1,034.60",NaN


In [3]:
# convert all the values from strings to floats. 
def string_to_float(x):
    if x is not np.nan:
        x = re.sub(",", "",x)
        x = float(x)
    else:
        x = 0 
    return x

for col in columns:
    if col != "Date":
        df[col] = df[col].apply(lambda x: string_to_float(str(x)))


In [4]:
df.tail()

,Date,Open,High,Low,Close_,AdjClose__,Volume
4,"Dec 08, 2020",6555.4,6567.6,6518.4,6558.8,6558.8,7.581868e+08
3,"Dec 09, 2020",6558.8,6623.0,6558.8,6564.3,6564.3,7.693453e+08
2,"Dec 10, 2020",6564.3,6640.7,6561.1,6599.8,6599.8,1.109877e+09
1,"Dec 11, 2020",6599.8,6607.4,6520.5,6546.8,6546.8,1.069086e+09
0,"Dec 14, 2020",6546.8,6583.7,6524.7,6531.8,6531.8,8.702654e+08


In [5]:
def add_moving_average(term, exp=False):
    for col in columns:
        if col != "Date" and col != "target":
            if exp:
                col_name = col + str(term) + "day" + "_exp"
                df[col_name] = df[col].rolling(window=term, win_type='triang').mean()
            else: 
                col_name = col + str(term) + "day"
                df[col_name] = df[col].rolling(window=term).mean()


In [6]:
flat_averages = [10,32,45,78,101]
exp_averages  = [3,12,48,112]

Run through and apply the moving averages to the columns 

In [7]:
for x in flat_averages: 
    add_moving_average(x)
for x in exp_averages:
    add_moving_average(x, exp=True)

In [8]:
len(df.columns)

61

In [9]:
df["target"] = df["Close_"].shift(1)

In [10]:
def word_target(x):
    y = "up"
    if x['Close_'] > x["target"]:
        y = "down"  
    return y 
df["wordT"] = df.apply(lambda x: word_target(x), axis=1)

In [11]:
df.drop("target", inplace=True, axis=1)

In [12]:
df.tail()

,Date,Open,High,Low,Close_,AdjClose__,Volume,Open10day,High10day,Low10day,...,Close_48day_exp,AdjClose__48day_exp,Volume48day_exp,Open112day_exp,High112day_exp,Low112day_exp,Close_112day_exp,AdjClose__112day_exp,Volume112day_exp,wordT
4,"Dec 08, 2020",6555.4,6567.6,6518.4,6558.8,6558.8,7.581868e+08,6426.40,6474.95,6388.26,...,6031.190538,6031.190538,9.713313e+08,5986.226339,6035.198724,5937.209774,5985.758450,5985.758450,7.577095e+08,down
3,"Dec 09, 2020",6558.8,6623.0,6558.8,6564.3,6564.3,7.693453e+08,6439.06,6490.38,6406.61,...,6052.891059,6052.891059,9.862015e+08,5985.758450,6034.844372,5936.970918,5985.649809,5985.649809,7.617105e+08,down
2,"Dec 10, 2020",6564.3,6640.7,6561.1,6599.8,6599.8,1.109877e+09,6456.38,6513.18,6427.77,...,6075.786632,6075.786632,1.000835e+09,5985.649809,6034.830931,5937.077455,5985.883913,5985.883913,7.657117e+08,down
1,"Dec 11, 2020",6599.8,6607.4,6520.5,6546.8,6546.8,1.069086e+09,6480.07,6536.33,6449.60,...,6099.417274,6099.417274,1.013642e+09,5985.883913,6035.150686,5937.526674,5986.465306,5986.465306,7.698038e+08,up
0,"Dec 14, 2020",6546.8,6583.7,6524.7,6531.8,6531.8,8.702654e+08,6497.99,6554.30,6475.45,...,6123.055208,6123.055208,1.023013e+09,5986.465306,6035.821110,5938.329225,5987.380517,5987.380517,7.739384e+08,up


Now comes the magic, maybe or some rubbish hard to say. The idea now is to convert the dataframe into a single string. 

In [13]:
big_string = ""

In [14]:

def add_to_string(x):
    global big_string
    if x == "\n":
        big_string += x 
    else:
        x = str(x) + " ; "
        big_string += x 

In [15]:
for i, row in df.iterrows():
    for col in df.columns:
        add_to_string(row[col])
    add_to_string("\n")


In [16]:
#print(big_string[0:10])

In [17]:
text = open("stock_text.txt", "w")
n = text.write(big_string)
text.close()